In [1]:
from ast import Param
from inspect import Parameter
from ax import (
    ComparisonOp,
    ParameterType,
    RangeParameter,
    ChoiceParameter,
    FixedParameter,
    SearchSpace,
    Experiment,
    OutcomeConstraint,
    OrderConstraint,
    SumConstraint,
    OptimizationConfig,
    Objective,
    Metric,
    Runner
)
from ax.metrics.l2norm import L2NormMetric
from ax.metrics.hartmann6 import Hartmann6Metric
from ax.modelbridge.registry import Models

Define parameter space

In [2]:
NUM_DIM = 6

hartmann_search_space = SearchSpace(
    parameters=[
        RangeParameter(
            name=f"x{i}", parameter_type=ParameterType.FLOAT, lower=0.0, upper=1.0
        )
        for i in range(NUM_DIM)
    ]
)

choice_param = ChoiceParameter(
    name="choice",
    values=["foo", "bar"],
    parameter_type=ParameterType.STRING,
    is_ordered=False,
    sort_values=False
)
fixed_param = FixedParameter(
    name="fixed",
    value=[True],
    parameter_type=ParameterType.BOOL
)

Define Constraints

In [3]:
sum_constraint = SumConstraint(
    parameters=[hartmann_search_space.parameters['x0'], hartmann_search_space.parameters['x1']],
    is_upper_bound=True,
    bound=5.0
)
order_constraint = OrderConstraint(
    lower_parameter=hartmann_search_space.parameters['x0'],
    upper_parameter=hartmann_search_space.parameters['x1']
)

Create optimization config

In [4]:
param_names = [f"x{i}" for i in range(NUM_DIM)]
optimization_config = OptimizationConfig(
    objective=Objective(
        metric=Hartmann6Metric(name="hartmann6", param_names=param_names),
        minimize=True
    ),
    outcome_constraints=[
        OutcomeConstraint(
            metric=L2NormMetric(
                name="l2norm", param_names=param_names, noise_sd=0.2
            ),
            op=ComparisonOp.LEQ,
            bound=1.25,
            relative=False
        )
    ]
)

Define a runner

In [5]:
class MyRunner(Runner):
    def run(self, trial):
        trial_metadata = {"name": str(trial.index)}
        return trial_metadata

Create experiment

In [6]:
exp = Experiment(
    name="test_hartmann",
    search_space=hartmann_search_space,
    optimization_config=optimization_config,
    runner=MyRunner()
)

Perform optimization

In [7]:
NUM_SOBOL_TRIALS = 5
NUM_BOTORCH_TRIALS = 5

print("Running Sobol initialization trials...")
sobol = Models.SOBOL(search_space=exp.search_space)

for i in range(NUM_SOBOL_TRIALS):
    # Produce a GeneratorRun from the model, which contains proposed 
    # arm(s) and other metadata
    generator_run = sobol.gen(n=1)
    # Add generator run to a trial to make it part of the epxeriment and
    # evaluate arm(s) in it
    trial = exp.new_trial(generator_run=generator_run)
    # Start trial run to evaluate arm(s) in the trial
    trial.run()
    # Mark trial as completed to record when a trial run is completed
    # and enable fetching of data for metrics on the experiment (by 
    # default, trials must be completed before metrics can fetch their 
    # data, unless a metric is explicitly configured otherwise)
    trial.mark_completed()

for i in range(NUM_BOTORCH_TRIALS):
    print(
        f"Running GP+EI optimization trial {i + NUM_SOBOL_TRIALS + 1}/{NUM_SOBOL_TRIALS + NUM_BOTORCH_TRIALS}..."
    )
    # Reinitialize GP+EI model at each step with updated data.
    gpei = Models.BOTORCH(experiment=exp, data=exp.fetch_data())
    generator_run = gpei.gen(n=1)
    trial = exp.new_trial(generator_run=generator_run)
    trial.run()
    trial.mark_completed()

print("Done!")

Running Sobol initialization trials...
Running GP+EI optimization trial 6/10...
Running GP+EI optimization trial 7/10...
Running GP+EI optimization trial 8/10...
Running GP+EI optimization trial 9/10...
Running GP+EI optimization trial 10/10...
Done!


In [8]:
trial_data = exp.fetch_trials_data([NUM_SOBOL_TRIALS + NUM_BOTORCH_TRIALS - 1])
trial_data.df

,arm_name,metric_name,mean,sem,trial_index,n,frac_nonnull
0,9_0,l2norm,1.230909,0.2,9,10000,1.230909
1,9_0,hartmann6,-2.082533,0.0,9,10000,-2.082533


In [9]:
exp.fetch_data().df

,arm_name,metric_name,mean,sem,trial_index,n,frac_nonnull
0,0_0,l2norm,0.737478,0.2,0,10000,0.737478
1,1_0,l2norm,1.984211,0.2,1,10000,1.984211
2,2_0,l2norm,1.327648,0.2,2,10000,1.327648
3,3_0,l2norm,1.429836,0.2,3,10000,1.429836
4,4_0,l2norm,1.322804,0.2,4,10000,1.322804
5,5_0,l2norm,1.176027,0.2,5,10000,1.176027
6,6_0,l2norm,0.994007,0.2,6,10000,0.994007
7,7_0,l2norm,1.113786,0.2,7,10000,1.113786
8,8_0,l2norm,1.402207,0.2,8,10000,1.402207
9,9_0,l2norm,1.120295,0.2,9,10000,1.120295


In [16]:
import numpy as np
from ax.plot.trace import optimization_trace_single_method
from ax.utils.notebook.plotting import render

objective_means = np.array([[trial.objective_mean for trial in exp.trials.values()]])
best_objective_plot = optimization_trace_single_method(
    y=np.minimum.accumulate(objective_means, axis=1),
    optimum=-3.32237
)
render(best_objective_plot)

In [20]:
exp.trials.values()

dict_values([Trial(experiment_name='test_hartmann', index=0, status=TrialStatus.COMPLETED, arm=Arm(name='0_0', parameters={'x0': 0.21083658933639526, 'x1': 0.669579803943634, 'x2': 0.010693836025893688, 'x3': 0.4864373207092285, 'x4': 0.0744972676038742, 'x5': 0.5185708999633789})), Trial(experiment_name='test_hartmann', index=1, status=TrialStatus.COMPLETED, arm=Arm(name='1_0', parameters={'x0': 0.8112051384523511, 'x1': 0.3749852441251278, 'x2': 0.9852933781221509, 'x3': 0.7664044266566634, 'x4': 0.815806488506496, 'x5': 0.16452474053949118})), Trial(experiment_name='test_hartmann', index=2, status=TrialStatus.COMPLETED, arm=Arm(name='2_0', parameters={'x0': 0.7306736595928669, 'x1': 0.9600422009825706, 'x2': 0.3201379394158721, 'x3': 0.21861496847122908, 'x4': 0.5855912063270807, 'x5': 0.44095097854733467})), Trial(experiment_name='test_hartmann', index=3, status=TrialStatus.COMPLETED, arm=Arm(name='3_0', parameters={'x0': 0.25535851903259754, 'x1': 0.00518511887639761, 'x2': 0.6853